# A) Initial Processing

### Create Snow Depth Rasters
Script to run terminal commands to create inititial snow depth grids. The process checks whether the rasters are the same size then will proceed to either A) warp the snow depth grid and subtract the bare earth or B) proceed to subtract the bare earth from snow depth. 
#### Troubleshooting
If issues with subprocess command, this can be run directly from the terminal. The "shell = True" solved my problems as I need to activate my anaconda environment to succesfully run GDAL commands. 

In [5]:
from osgeo import gdal
import numpy
import os
import rasterio as rio
import subprocess
import sys
from subprocess import Popen, PIPE, CalledProcessError

In [21]:
#function to subtract the bare earth from the snow on raster to create snow depth grid
def gdal_calc(bareearth_raster, snowon_raster, gdalcalc,snowdepth_raster):
     #if this file already exists, gdal_calc will not run 
    if os.path.exists(snowdepth_raster): 
        os.remove(snowdepth_raster)
    cmd_calc = 'python {} -A {} -B {} --outfile={} --calc="A-B"'.format(gdalcalc, snowon_raster, bareearth_raster,snowdepth_raster)
    #this will print the gdal calc processing
    with Popen(cmd_calc, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here
    subprocess.run(cmd_calc, shell=True) #should exit with code 0 if succesful 

In [8]:
#function to first warp the "snowon_raster" to the bare earth if different extents
def gdal_warp(bareearth_raster, snowon_raster, be_read):
    #create the file name by adding warp to the snowon raster file name
    print("creating bounding box..")
    min_x=be_read.bounds[0]
    max_x=be_read.bounds[2]
    min_y=be_read.bounds[1]
    max_y=be_read.bounds[3]
    warp_name= str(snowon_raster)[:-4] + "_warp.tif"
    print(warp_name)
    #if this file already exists, gdawarp will not run 
    if os.path.exists(warp_name): 
        os.remove(warp_name)
    #creating command statement   
    cmd_warp = 'gdalwarp -te {} {} {} {} {} {}'.format(min_x, min_y, max_x, max_y, snowon_raster, warp_name)
    #enabling subprocess to print out live processing
    with Popen(cmd_warp, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout: 
            print(line, end='')
    #call the gdal_warp command
    subprocess.run(cmd_warp, shell=True)
    #return the output file
    return warp_name

In [22]:
#set watershed 
watershed = "Tsitika"
wshed = "TSI"
phase = "P01"
year = "21"

path_parent = os.path.dirname(os.getcwd())
rasters = os.path.join(path_parent, watershed, "5_lidar_processing","datasets","rasters")
print(rasters)

#file directrories
bareearth_raster = os.path.join(rasters, "{}_bare_earth.tif".format(wshed))
snowon_raster = os.path.join(rasters,"snowon_coreg", "{}_{}_slave.tif".format(wshed,phase))
snowdepth_raster = os.path.join(rasters,"snowdepth_raw", "{}_{}_{}_SD_raw.tif".format(wshed,phase,year))
print(snowdepth_raster)
gdalcalc = "gdal_calc.py"


H:\ACO\2021\Tsitika\5_lidar_processing\datasets\rasters
H:\ACO\2021\Tsitika\5_lidar_processing\datasets\rasters\snowdepth_raw\TSI_P01_SD_raw.tif


In [23]:
#based on snow_on raster name, may need to update if your naming convention differs
print(snowdepth_name)

#read in the rasters
be_read = rio.open(bareearth_raster)
snowon_read = rio.open(snowon_raster)

#generate a hillshade for the snowon raster


#Check if the rasters are different sizes
print("bareearth size: " + str(be_read.shape))
print("snowon size: " + str(snowon_read.shape))
if be_read.shape != snowon_read.shape:
    #Warp the snowon raster then reassign the variable
    snowon_raster = gdal_warp(bareearth_raster, snowon_raster, be_read)

#subtract the bare earth from the snow on raster and save in outputs folder
print("subtracting rasters..")
gdal_calc(bareearth_raster, snowon_raster, gdalcalc,snowdepth_raster)

TSI_P01_SD_raw.tif
bareearth size: (6088, 5476)
snowon size: (6088, 5476)
subtracting rasters..
0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 